In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for visualising

In [2]:
# read in all the words
words = open("data.txt", encoding="utf-8").read().splitlines()
words = [i.strip() for i in words if i]

In [3]:
len(words)

905

In [6]:
# build the vocabulary of characters and mappings to /from integers
chars = sorted(list(set(''.join(words)))) # getting the character library

stoi = {val:key+1 for key, val in enumerate(chars)}
stoi['|'] = 0
itos = {ind:st for st, ind in stoi.items()}
print(itos)

{1: '-', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e', 7: 'f', 8: 'g', 9: 'h', 10: 'i', 11: 'j', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'q', 19: 'r', 20: 's', 21: 't', 22: 'u', 23: 'v', 24: 'w', 25: 'x', 26: 'y', 27: 'z', 28: 'é', 29: 'ë', 30: 'í', 31: 'ó', 32: 'ú', 33: 'û', 34: '’', 0: '|'}


In [7]:
# building the dataset

block_size = 3 # context length: how many characters we take to predict the next one
X, Y = [], []

for w in words[:5]:
    print(w)
    context = [0] * block_size

    for ch in w + '|':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

aego
||| ---> a


TypeError: can only concatenate list (not "int") to list